# 금속 주조 불량 판정 모델
### <분석 목표>
- 공정 환경과 공정 변수를 관리, 불량에 대응하여 공정 최적화 진행  
- 불량 판별하는 모델 구축하여 생산성 향상 추진 및 비용 절감 기대


In [ ]:
# 필요한 패키지 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

## 데이터 불러오기 및 데이터 기본 정보 확인하기

In [ ]:
df = pd.read_csv('./defect.csv', encoding = 'cp949')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

## 데이터 전처리


In [ ]:
# 컬럼명 변경
df.columns = ['Unnamed:0', '작업라인', '제품명', '금형명', '수집날짜', '수집시각', '일자별제품생산번호',
              '가동여부', '비상정지', '용탕온도', '설비작동사이클시간', '제품생산사이클시간',
              '저속구간속도', '고속구간속도', '용탕량', '주조압력', '비스킷두께', '상금형온도1',
              '상금형온도2', '상금형온도3', '하금형온도1', '하금형온도2', '하금형온도3', '슬리브온도',
              '형체력', '냉각수온도', '전자교반가동시간', '등록일시', '불량판정', '사탕신호', '금형코드',
              '가열로']

In [ ]:
# 결측치 비율 확인
df['불량판정'].value_counts()
len(df[df['불량판정'] == 0]) /len(df) * 100
len(df[df['불량판정'] == 1]) / len(df) * 100

In [ ]:
# 결측치 값이 30% 이상인 데이터들은 데이터의 완전성이 떨어지기 때문에 삭제
df = df[df.isnull().mean(axis = 1) * 100 <= 30]

- 사탕신호가 'D'로 측정될 때, 해당 생산품은 전부 불량 판정(=1)
- 해당 생산품은 다른 설명 변수를 고려하지 않고 사탕 신호로만 불량 여부 설명을 했다고 판단
- 따라서, 'D'인 행들은 분석에서 고려하지 않기로 결정

In [ ]:
import koreanize_matplotlib
# NaN 값을 0으로 변환하고, 'D'는 1로 변환하는 코드 실행
df['사탕신호'] = df['사탕신호'].replace('D', 1)  # 'D'를 1로 변환
df['사탕신호'] = df['사탕신호'].fillna(0)  # NaN을 0으로 변환

# 사탕신호의 unique 값 확인
df['사탕신호'].unique()

# 사탕신호가 1일 때 불량판정이 0인 비율과 1인 비율을 계산
filtered_df = df[df['사탕신호'] == 1]
filtered_df_zero = df[df['사탕신호'] == 0]

# 불량판정의 비율 계산
#passorfail_0_ratio = (filtered_df['불량판정'].value_counts(normalize=True)[0] * 100).round(2)
#사탕신호 1인 것 중 실제 불량이 0인 비율 0%라 오류 뜸

passorfail_0_ratio = 0.0
#사탕신호 1인 것 중 실제 불량이 0인 비율은 0.0으로 수동 대체

passorfail_1_ratio = (filtered_df['불량판정'].value_counts(normalize=True)[1] * 100).round(2)
#사탕신호 1인 것 중 실제 불량이 1인 비율 100%

passorfail_0_ratio_zero = (filtered_df_zero['불량판정'].value_counts(normalize=True)[0] * 100).round(2)
#사탕신호 0인 것 중 실제 불량도 0인 비율 97.67%

passorfail_1_ratio_zero = (filtered_df_zero['불량판정'].value_counts(normalize=True)[1] * 100).round(2)
#사탕신호 0인 것 중 실제 불량이 1인 비율 2.33%

# 비율 데이터를 리스트로 저장
labels = ['NaN', 'D']
fail_0_ratios = [passorfail_0_ratio_zero, passorfail_0_ratio]  # 불량판정 0 비율
fail_1_ratios = [passorfail_1_ratio_zero, passorfail_1_ratio]  # 불량판정 1 비율

x = np.arange(len(labels))  # 라벨의 개수만큼 x 위치 생성
width = 0.35  # 막대 너비

fig, ax = plt.subplots(figsize=(8, 6))

# 불량판정 0 막대 그래프 그리기
bars1 = ax.bar(x - width/2, fail_0_ratios, width, label='정상', color='#7FB3D5')

# 불량판정 1 막대 그래프 그리기
bars2 = ax.bar(x + width/2, fail_1_ratios, width, label='불량', color='#F1948A')

# 각각의 막대에 % 비율 추가
for bar in bars1:
    ax.annotate(f'{bar.get_height():.2f}%',
                (bar.get_x() + bar.get_width() / 2, bar.get_height()),
                ha='center', va='bottom')

for bar in bars2:
    ax.annotate(f'{bar.get_height():.2f}%',
                (bar.get_x() + bar.get_width() / 2, bar.get_height()),
                ha='center', va='bottom')

# 그래프 레이블 설정
ax.set_ylabel('불량율 (%)')
ax.set_title('사탕신호에 따른 불량판정 비율 비교')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.0), ncol=2)

# 그래프 레이아웃 조정 및 출력
plt.tight_layout()
plt.show()

In [ ]:
# '사탕신호'가 'D'인 행 Drop
df = df[df['사탕신호'] != 1]

- 사용하지 않을 열 드롭
- '등록일시' datetime 형식으로 데이터 타입 변경
- 범주형 데이터 타입 변경

In [ ]:
# 열 Drop (등록일시 포함)
eda_df = df.copy()
eda_df =  eda_df.drop(columns=['Unnamed:0', '일자별제품생산번호', '작업라인', '사탕신호', '제품명', '금형명', '비상정지', '수집날짜', '수집시각'])
df = df.drop(columns=['Unnamed:0', '일자별제품생산번호', '작업라인', '사탕신호', '제품명', '금형명', '비상정지', '수집날짜', '수집시각', '등록일시'])

# '불량판정', '금형코드' 열을 범주형으로 변환

df['불량판정'] = df['불량판정'].astype('category')
df['금형코드'] = df['금형코드'].astype('category')

eda_df['불량판정'] = eda_df['불량판정'].astype('category')
eda_df['금형코드'] = eda_df['금형코드'].astype('category')

df.isna().sum()

## 기초통계량 보기
1. 불량(1)인 데이터들의 온도(상금형온도1, 상금형온도2, 하금형온도1, 하금형온도2)의 평균과 중앙값이 낮음.
2. 불량(1)인 데이터들의 주조압력 평균과 중앙값이 낮음.

In [ ]:
numeric_df = df[['용탕온도', '설비작동사이클시간', '제품생산사이클시간',
              '저속구간속도', '고속구간속도', '용탕량', '주조압력', '비스킷두께', '상금형온도1',
              '상금형온도2', '상금형온도3', '하금형온도1', '하금형온도2', '하금형온도3', '슬리브온도',
              '형체력', '냉각수온도', '전자교반가동시간']]
category_df = df[['가동여부', '금형코드', '가열로']]

In [ ]:
numeric_df.describe()

In [ ]:
defect_df = numeric_df[df['불량판정']==1]
defect_df.describe()

In [ ]:
defect_df = numeric_df[df['불량판정']==0]
defect_df.describe()

## 시각화
### Target: 불량판정

In [ ]:
# 불량판정의 카운트 값 계산
counts = df['불량판정'].value_counts()

# '불량' 판정을 강조하기 위해 분리 (불량이 '1.0'이라고 가정)
explode = [0.1 if label == '1' else 0 for label in counts.index]  # '1.0' 부분을 강조

# 화려한 색상 팔레트 (색상 조정)
colors = sns.color_palette('coolwarm', len(counts))

# 파이 차트 그리기 (불량 조각을 빼냄)
plt.figure(figsize=(6, 6))
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', explode=explode, colors=colors, wedgeprops={'edgecolor': 'none'})
plt.title('사탕신호 "D" 행 제거 후 불량판정 분포', fontsize=15)
plt.show()

### 수치형 변수 시각화
- 주조압력, 상금형온도1, 하금형온도1, 하금형온도2의 불량판정에 따른 분포가 상이한 것을 확인할 수 있음.

In [ ]:
# 수치형 변수들에 대한 박스플롯을 서브플롯으로 그리는 함수
def boxplot_subplots(df):
    # 수치형 변수 선택 (int64와 float64 타입만 선택)
    numeric_cols = numeric_df.columns

    # 불량판정을 범주형 변수로 변환 (안전성을 위해 처리)
    if '불량판정' in df.columns:
        df['불량판정'] = df['불량판정'].astype(str)

    # 서브플롯 행 계산 (한 열에 두 개의 서브플롯 배치)
    total_plots = len(numeric_cols) * 2  # 한 변수당 2개의 플롯
    max_cols = 2  # 한 행에 2개의 서브플롯
    max_rows = int(np.ceil(total_plots / max_cols))  # 행 개수 계산

    plt.figure(figsize=(15, 6 * max_rows))  # 전체 플롯 크기 설정

    plot_index = 1
    for col in numeric_cols:
        # 첫 번째 서브플롯: 불량판정 기준으로 박스플롯
        plt.subplot(max_rows, max_cols, plot_index)
        sns.boxplot(data=df, x='불량판정', y=col, palette='Set2')
        plt.title(f'{col}의 불량판정별 박스플롯', fontsize=12)
        plt.xlabel('불량판정', fontsize=10)
        plt.ylabel(col, fontsize=10)
        plt.xticks(rotation=90)
        plot_index += 1

        # 두 번째 서브플롯: 전체 데이터를 대상으로 박스플롯
        plt.subplot(max_rows, max_cols, plot_index)
        sns.boxplot(data=df, y=col)
        plt.title(f'{col}의 전체 데이터 박스플롯', fontsize=12)
        plt.ylabel(col, fontsize=10)
        plt.xticks(rotation=90)
        plot_index += 1

    plt.tight_layout()  # 서브플롯 간의 간격 조정
    plt.show()

# 박스플롯 그리기
boxplot_subplots(df)

## 일별 분량판정률 변화
- 1월 2일, 1월 27일, 2월 12일의 불량판정평균이 높음

In [ ]:
# 불량판정 열을 float으로 변환
eda_df['불량판정'] = pd.to_numeric(eda_df['불량판정'], errors='coerce')
# 불량 판정의 일별 불량률 계산
def plot_daily_failure_rate(df, date_col, fail_col):
    # '등록일시' 열을 datetime 형식으로 변환
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

    # '불량판정' 열을 숫자형(float)으로 변환
    df[fail_col] = pd.to_numeric(df[fail_col], errors='coerce')

    # 일별로 그룹화하여 불량률 계산
    daily_failure_rate = df.groupby(df[date_col].dt.date)[fail_col].mean().reset_index()
    daily_failure_rate.columns = ['Date', 'Failure Rate']  # 새로운 열 이름 설정

    # 그래프 그리기
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=daily_failure_rate, x='Date', y='Failure Rate', marker='o', color='orange')

    # 그래프 설정
    plt.title('일별 불량판정률 변화', fontsize=15)
    plt.xlabel('날짜', fontsize=12)
    plt.ylabel('불량률', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True)

    # 그래프 출력
    plt.tight_layout()
    plt.show()

    return daily_failure_rate  # 일별 불량률 데이터 반환

# 상위 n개의 불량률이 높은 날짜를 추출하는 함수
def find_top_n_failure_dates(df, n=3):
    # 불량률이 높은 순서대로 상위 n개 데이터 추출
    top_n = df.nlargest(n, 'Failure Rate')
    return top_n

# 일별 불량률 계산 및 그래프 그리기
daily_failure_rate = plot_daily_failure_rate(eda_df, '등록일시', '불량판정')

# 상위 3개의 날짜 추출
top_3_dates = find_top_n_failure_dates(daily_failure_rate, n=3)
print(top_3_dates)

### 1월 27일 수치형 변수들과 불량률 라인 그래프
- 설비작동사이클시간이 길 때 불량률 높은 경향을 보임
- 주조압력이 낮을 때 불량률 높은 경향을 보임
- 저속구간속도가 낮을 때 불량률 높은 경향을 보임
- 슬리브온도가 낮을 때 불량률 높은 경향을 보임
- 상금형온도1,2가 낮을 때 불량률이 높은 경향을 보임
- 하금형온도1,2가 낮을 때 불량률이 높은 경향을 보임

In [ ]:
# 불량률과 수치형 변수 겹쳐 그리기
def plot_numeric_variables_and_defect_rate(df, defect_col, time_col, numeric_cols, date):
    # 특정 날짜에 해당하는 데이터 필터링
    df[time_col] = pd.to_datetime(df[time_col])
    daily_data = df[df[time_col].dt.date == pd.to_datetime(date).date()]

    # 불량률 계산 (불량 판정 열이 1일 때 불량으로 간주)
    daily_data['불량률'] = daily_data[defect_col].rolling(window=10, min_periods=1).mean()

    # 서브플롯 그리기 (한 줄에 2개의 그래프씩)
    num_vars = len(numeric_cols)
    fig, axes = plt.subplots(num_vars, 1, figsize=(12, 4 * num_vars), sharex=True)

    if num_vars == 1:
        axes = [axes]  # 1개일 경우 리스트로 변환

    for i, col in enumerate(numeric_cols):
        ax = axes[i]

        # 수치형 변수의 선 그래프
        ax.plot(daily_data[time_col], daily_data[col], label=col, color='blue')
        ax.set_ylabel(col)

        # 불량률 선 그래프 (이중 y축)
        ax2 = ax.twinx()
        ax2.plot(daily_data[time_col], daily_data['불량률'], label='불량률', color='orange', alpha=0.7, linestyle='--')
        ax2.set_ylabel('불량률')
        ax2.set_ylim(0, 1)  # 불량률은 0에서 1 사이로 고정

        # 범례 추가
        ax.legend(loc='upper left')
        ax2.legend(loc='upper right')

    # X축에 시간 표시
    plt.xlabel('시간')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# 예시 사용법
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
plot_numeric_variables_and_defect_rate(eda_df, '불량판정', '등록일시', numeric_cols, '2019-01-27')


## 전자교반가동시간 시각화

In [ ]:
sns.countplot(data = eda_df, x = '전자교반가동시간', hue = '불량판정')

## 범주형 변수 시각화 및 가설검정
- 가동 여부와 금형 코드에 따라 불량 판정이 달라짐
- 가열로와 불량 판정은 서로 독립적


### 가동여부

In [ ]:
df['가동여부'].value_counts()
pivot_working = df.pivot_table(index='가동여부', columns='불량판정', aggfunc='size', fill_value=0)
pivot_working

In [ ]:
sns.countplot(data = eda_df, x = '가동여부', hue = '불량판정', palette='Blues')
# 그래프 제목 및 라벨 설정
plt.title('가동여부별 불량판정 Barplot')
plt.xlabel('가동여부')
plt.ylabel('Count')
plt.legend(title='불량판정')
plt.show()

In [ ]:
# 교차표 생성
cross_tab = pd.crosstab(df['가동여부'], df['불량판정'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"Chi-square statistic: {chi2}")
print(f"p-value: {p}")

대립가설 채택, 가동여부에 따라 불량판정이 달라짐

### 금형코드

In [ ]:
pivot_code = df.pivot_table(index = '금형코드', columns = '불량판정', aggfunc = 'size', fill_value=0)
pivot_code

In [ ]:
# 피벗 테이블을 barplot에 맞게 다시 변환
pivot_reset = pivot_code.reset_index().melt(id_vars='금형코드', value_vars=['0.0', '1.0'], var_name='불량판정', value_name='Count')

# barplot 그리기
plt.figure(figsize=(10,6))
sns.barplot(data=pivot_reset, x='금형코드', y='Count', hue='불량판정', palette='Blues')

# 그래프 제목 및 라벨 설정
plt.title('금형코드별 불량판정 Barplot')
plt.xlabel('금형코드')
plt.ylabel('Count')
plt.legend(title='불량판정')
plt.show()

In [ ]:
# 교차표 생성
cross_tab = pd.crosstab(df['금형코드'], df['불량판정'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"Chi-square statistic: {chi2}")
print(f"p-value: {p}")

대립가설 채택, 금형코드에 따라 불량판정이 달라짐

### 가열로

In [ ]:
df['가열로'].value_counts()
df['가열로'] = df['가열로'].fillna('F')
pivot_heating = df.pivot_table(index='가열로', columns='불량판정', aggfunc='size', fill_value=0)
pivot_heating

In [ ]:
# 피벗 테이블을 barplot에 맞게 다시 변환
pivotreset_heating = pivot_heating.reset_index().melt(id_vars='가열로', value_vars=['0.0', '1.0'], var_name='불량판정', value_name='Count')

# barplot 그리기
plt.figure(figsize=(10,6))
sns.barplot(data=pivotreset_heating, x='가열로', y='Count', hue='불량판정', palette='Blues')

# 그래프 제목 및 라벨 설정
plt.title('가열로별 불량판정 Barplot')
plt.xlabel('가열로')
plt.ylabel('Count')
plt.legend(title='불량판정')
plt.show()

In [ ]:
# 교차표 생성
cross_tab = pd.crosstab(df['가열로'], df['불량판정'])

# 카이제곱 검정 수행
chi2, p, dof, expected = chi2_contingency(cross_tab)

# 결과 출력
print(f"Chi-square statistic: {chi2}")
print(f"p-value: {p}")

귀무가설 채택, 가열로와 불량판정은 서로 독립

## 전처리
### 결측치 처리

In [ ]:
# 가열로, 용탕온도, 용탕량, 하금형온도, 상금형온도3에서 결측치 발견
df.isna().sum()

### 용탕온도, 용탕량, 하금형온도3에 대해 선형 보간 적용

In [ ]:
# 선형보간용 df2를 만듦
df2 = df.copy()

# 실제값, NaN, NaN, 실제값이 나오는 인덱스 출력
df2['상금형온도3'][634:638]

# 인덱스를 리셋해서 숫자형으로 만듦
df_reset = df2.reset_index(drop=True)

# 해당 구간 추출 (634~637 사용)
subset_df = df_reset.loc[634:637, ['상금형온도3']]

# 선형 보간 적용
subset_df['상금형온도3'] = subset_df['상금형온도3'].interpolate(method='linear')

# 보간된 결과 출력
subset_df

# 보간 전
#634   150.000000
#635   NaN
#636   NaN
#637   124.000000

# 보간 후
#634   150.000000
#635   141.333333
#636   132.666667
#637   124.000000

# 시각화
plt.figure(figsize=(8, 6))

# 선형 보간된 값을 선으로 연결하여 시각화
plt.plot(subset_df.index, subset_df['상금형온도3'], marker='o', linestyle='-', color='blue', label='선형보간 이후')

# NaN이었던 값들 강조 (보간된 값 표시)
plt.scatter([635, 636], subset_df.loc[[635, 636], '상금형온도3'], color='red', label='보간된 NaN', zorder=5)

# 그래프 축을 데이터 범위에 맞게 설정
plt.ylim(subset_df['상금형온도3'].min() - 10, subset_df['상금형온도3'].max() + 10)

# 라벨 설정
plt.xlabel('인덱스')
plt.ylabel('상금형온도3')
plt.title('상금형온도3 보간 후 비교 (NaN 강조)')

# 범례 추가
plt.legend()

# 그래프 출력
plt.show()

In [ ]:
df['용탕온도'] = df['용탕온도'].interpolate(method='linear', limit_direction='both')
df['용탕량'] = df['용탕량'].interpolate(method='linear', limit_direction='both')
df['하금형온도3'] = df['하금형온도3'].interpolate(method='linear', limit_direction='both')
df['상금형온도3'] = df['상금형온도3'].interpolate(method='linear', limit_direction='both')
df.isna().sum()

## 이상치 제거

In [ ]:
# 수치형 변수들에 대한 박스플롯을 서브플롯으로 그리는 함수
def boxplot_subplots(df):
    # 수치형 변수 선택 (int64와 float64 타입만 선택)
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

    # 서브플롯 행 계산 (한 행에 2개의 서브플롯 배치)
    total_plots = len(numeric_cols)  # 각 변수당 1개의 플롯
    max_cols = 2  # 한 행에 2개의 서브플롯
    max_rows = int(np.ceil(total_plots / max_cols))  # 행 개수 계산

    plt.figure(figsize=(15, 6 * max_rows))  # 전체 플롯 크기 설정

    plot_index = 1
    for col in numeric_cols:
        # 각 변수에 대해 하나의 박스플롯 그리기
        plt.subplot(max_rows, max_cols, plot_index)
        sns.boxplot(data=df, y=col, palette='Set2')
        plt.title(f'{col}의 박스플롯', fontsize=12)
        plt.ylabel(col, fontsize=10)
        plt.xticks(rotation=90)
        plot_index += 1

    plt.tight_layout()  # 서브플롯 간의 간격 조정
    plt.show()

# 박스플롯 그리기
boxplot_subplots(df)

In [ ]:
df = df[df['설비작동사이클시간'] <= 400] # 1
df = df[df['제품생산사이클시간'] <= 450] # 2
df = df[df['저속구간속도'] <= 60000] # 1
df = df[df['상금형온도1'] <= 1400] # 1
df = df[df['상금형온도2'] <= 4000] # 1
df = df[df['하금형온도3'] <= 60000] # 1
df = df[df['형체력'] <= 60000] # 3
df = df[df['냉각수온도'] <= 1400] # 9

### 변수 간 상관관계 확인하기

In [ ]:
def plot_spearman_corr(df):
    # 수치형 변수만 선택
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

    # 스피어만 상관계수 계산
    spearman_corr = df[numeric_cols].corr(method='spearman')

    # 히트맵으로 상관계수 시각화
    plt.figure(figsize=(10, 8))
    sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    plt.title('스피어만 순위 상관계수 히트맵')
    plt.show()

# 함수 호출
plot_spearman_corr(df)


## IV(Information Value)

In [ ]:
# 랜덤 시드 설정
np.random.seed(42)
df['불량판정'] = pd.to_numeric(df['불량판정'], errors='coerce')

### 주조압력 IV

In [ ]:
def 동일_데이터_구간_나누기(df, column, num_bins=10):
    # 데이터 수 동일한 구간으로 나누기 (중복 허용)
    df['구간'] = pd.qcut(df[column].rank(method='first'), q=num_bins, labels=False)

    # 각 구간에 대한 통계 정보 계산
    grouped = df.groupby('구간').apply(lambda x: pd.Series({
        '데이터 건수': len(x),
        '불량판정 0 개수': (x['불량판정'] == 0).sum(),
        '불량판정 1 개수': (x['불량판정'] == 1).sum()
    })).reset_index()

    # WOE와 IV 계산
    total_good = df['불량판정'].value_counts()[0]
    total_bad = df['불량판정'].value_counts()[1]

    # 각 구간의 비율 계산
    grouped['불량판정 0 비율'] = grouped['불량판정 0 개수'] / total_good
    grouped['불량판정 1 비율'] = grouped['불량판정 1 개수'] / total_bad

    # WOE 계산
    grouped['WOE'] = np.log(grouped['불량판정 1 비율'] / grouped['불량판정 0 비율'].replace(0, np.nan))

    # IV 계산
    grouped['IV'] = (grouped['불량판정 1 비율'] - grouped['불량판정 0 비율']) * grouped['WOE']
    iv_value = grouped['IV'].sum()  # 전체 IV 값

    # 수치 범위 추가
    grouped['수치 범위'] = grouped.apply(lambda x: f"{df[df['구간'] == x['구간']][column].min()} - {df[df['구간'] == x['구간']][column].max()}", axis=1)

    # 수치 범위를 첫 번째 열로 이동
    grouped = grouped[['수치 범위', '구간', '데이터 건수', '불량판정 0 개수', '불량판정 1 개수',
                       '불량판정 0 비율', '불량판정 1 비율', 'WOE', 'IV']]

    return grouped, iv_value

result, total_iv = 동일_데이터_구간_나누기(df, '주조압력', num_bins=10)
result

### 전체 변수 IV
- 공정 최적화를 위해 관리해야할 IV 값 상위 3개 변수를 선정: 주조압력, 하금형온도2, 하금형온도1

In [ ]:
# IV 계산 함수 (범주형 및 수치형 변수를 모두 처리)
def calculate_iv(df, target, cat_cols):
    iv_dict = {}

    # 타겟 변수를 숫자로 변환 (필요한 경우)
    df[target] = pd.to_numeric(df[target], errors='coerce')  # 불량판정 열을 숫자형으로 변환

    total_events = df[target].sum()  # 타겟이 1인 경우의 총합 (이벤트)
    total_non_events = df[target].count() - total_events  # 타겟이 0인 경우의 총합 (비이벤트)

    # 범주형 변수 처리
    for col in cat_cols:
        # 각 범주별 이벤트와 비이벤트 수 계산
        grouped = df.groupby(col)[target].agg(['count', 'sum']).reset_index()
        grouped.columns = [col, 'total', 'events']

        # 비이벤트 수 계산
        grouped['non_events'] = grouped['total'] - grouped['events']

        # WOE 및 IV 계산
        grouped['event_rate'] = grouped['events'] / total_events
        grouped['non_event_rate'] = grouped['non_events'] / total_non_events
        grouped['WOE'] = np.log(grouped['event_rate'] / grouped['non_event_rate']).replace([np.inf, -np.inf], 0)  # 무한대 처리
        grouped['IV'] = (grouped['event_rate'] - grouped['non_event_rate']) * grouped['WOE']

        # IV 값 계산
        iv = grouped['IV'].sum()
        iv_dict[col] = iv

    # 수치형 변수 처리
    for col in df.select_dtypes(include=[np.number]).columns:
        if col == target:  # 타겟 변수를 제외
            continue

        # 데이터를 10개 구간으로 나누기
        df['ranked'] = df[col].rank(method='first')
        df['bin'] = pd.qcut(df['ranked'], 10, labels=False, duplicates='drop')

        # 각 구간에 대한 이벤트와 비이벤트 수 계산
        grouped = df.groupby('bin')[target].agg(['count', 'sum']).reset_index()
        grouped.columns = ['bin', 'total', 'events']

        # 비이벤트 계산
        grouped['non_events'] = grouped['total'] - grouped['events']

        # WOE 및 IV 계산
        grouped['event_rate'] = grouped['events'] / total_events
        grouped['non_event_rate'] = grouped['non_events'] / total_non_events
        grouped['WOE'] = np.log(grouped['event_rate'] / grouped['non_event_rate']).replace([np.inf, -np.inf], 0)
        grouped['IV'] = (grouped['event_rate'] - grouped['non_event_rate']) * grouped['WOE']

        # IV 값 계산
        iv = grouped['IV'].sum()
        iv_dict[col] = iv

    return iv_dict

# 범주형 변수 리스트
cat_cols = ['가동여부', '가열로', '금형코드']

# IV 값 계산
iv_values = calculate_iv(df, '불량판정', cat_cols)

# 결과 출력
iv_values_df = pd.DataFrame(list(iv_values.items()), columns=['Variable', 'IV'])
iv_values_df = iv_values_df.sort_values(by='IV', ascending=False)
iv_values_df


### IV 상위 3개 변수 불량률 계산

In [ ]:
def 구간별_불량_통계(df, column):
    # 최소값과 최대값 기준으로 10개 구간 정의
    min_value = df[column].min()
    max_value = df[column].max()
    bins = np.linspace(min_value, max_value, num=11)  # 10개 구간을 나누기 위해 11개의 경계 값 생성

    # 수치형 변수를 10개 구간으로 나누기
    df['구간'] = pd.cut(df[column], bins=bins, include_lowest=True)

    # 각 구간에 대한 통계 정보 계산
    grouped = df.groupby('구간').apply(lambda x: pd.Series({
        '데이터 수': len(x),
        '불량 갯수 (1)': (x['불량판정'] == 1).sum()
    })).reset_index()

    # 불량률 계산
    grouped['불량률'] = grouped['불량 갯수 (1)'] / grouped['데이터 수'] * 100
    grouped['불량률'] = grouped['불량률'].round(3)
    grouped['불량률'] = grouped['불량률'].astype(str) + '%'

    # 수치 범위 추가
    grouped['수치 범위'] = grouped['구간'].astype(str)

    # 최종 결과 정리
    결과 = grouped[['수치 범위', '데이터 수', '불량 갯수 (1)', '불량률']]

    return 결과

result1 = 구간별_불량_통계(df, '주조압력')
result1
result2 = 구간별_불량_통계(df, '상금형온도2')
result2
result3 = 구간별_불량_통계(df, '하금형온도2')
result3

In [ ]:
result1 # 주조압력

### 전체 조건 만족 시 불량률의 감소율

In [ ]:
num= df.shape[0] # 기존 데이터 90076개
num_def = df['불량판정'].value_counts()[1] # 기존 불량 2087
def_rate = round((num_def / num) * 100, 3) # 기존 불량률 2.317%

df_opt = df.query('주조압력>286.6 & 상금형온도2>127.2 & 상금형온도2<=239.4 & 하금형온도2>117.6 & 하금형온도2<=310.8')
num_opt = df_opt.shape[0] # 최적화 후 데이터 77412개
num_opt_def = df_opt['불량판정'].value_counts()[1]  # 최적화 후 불량 397개
def_rate_opt = round((num_opt_def / num_opt) * 100, 3) # 최적화 후 불량률 0.513%

dec_rate = round(((def_rate - def_rate_opt) / def_rate) * 100, 3)# 불량률의 감소율 77.859%

In [ ]:
labels = ['전', '후']
values = [def_rate, def_rate_opt]

plt.figure(figsize=(6, 4))
plt.bar(labels, values, color=['pink', 'skyblue'])

plt.title('공정 최적화 전후 불량률')
plt.ylabel('불량률 (%)')

plt.ylim(0, max(def_rate, def_rate_opt) + 1)

for i, v in enumerate(values):
    plt.text(i, v + 0.1, f'{v:.3f}%', ha='center')

plt.show()

## Decision Tree

In [ ]:
df = df.drop(['구간', 'ranked', 'bin'], axis = 1)

In [ ]:
# '가동여부' 변환: 가동이면 0, 아니면 1
df['가동여부'] = df['가동여부'].apply(lambda x: 0 if x == '가동' else 1)


# '가열로', '금형코드' 칼럼 더미변수로 인코딩
df = pd.get_dummies(df, columns = ["가열로", "금형코드"], drop_first = True)

In [ ]:
# 변수 중요도 시각화
# model fit을 위한 X, y 분류
X = df.drop("불량판정", axis = 1)
y = df["불량판정"]

# Decision Tree 모델 학습
dt_model = DecisionTreeClassifier(random_state= 42, max_depth=3)

dt_model.fit(X, y)
plt.figure(figsize=(10, 6))
importances = dt_model.feature_importances_
indices = np.argsort(importances)[::-1]
features = X.columns

# 한글 폰트 설정
plt.title('Feature Importances (Decision Tree)')
sns.barplot(x=importances[indices], y=[features[i] for i in indices], palette="coolwarm")
plt.tight_layout()
plt.show()

In [ ]:
# 트리 시각화
from sklearn.tree import plot_tree
plt.figure(figsize=(20, 10))  # 그림 크기 설정
plot_tree(dt_model, filled=True, feature_names=X.columns, class_names=['Good', 'Defective'], rounded=True)
plt.show()